# Find Addresses for Parcels from Patriot

## Import modules and declare globals

In [ ]:
#Use arcNew environment with this notebook.
#Current Location: C:\ProgramData\Anaconda3\envs\arcNew\python.exe
import arcpy as arc
import pandas as pd
import numpy as np
import os
from shutil import copy2
from pathlib import Path
from arcgis import features
from datetime import datetime as dt
from time import sleep

## Create Folders in the Users Directory since it should have r/w permissions for the user.

In [ ]:
arc.env.overwriteOutput = True
arc.env.outputZFlag = 'Disabled' #To remove z data from parcel fabric due to it being a polygonZ
arc.env.outputMFlag = 'Disabled'
arc.env.qualifiedFieldNames = False

now = dt.now()
mStr = now.strftime('%m%Y')
dStr = now.strftime('%m_%d')
uPath = Path.home()
locFolders = ['Processing', 'Review']
if uPath.exists():
    for x in locFolders:
        a = Path(uPath / 'GIS' / x)
        if a.exists():
            print(f'{a} already exists.')
        else:
            a.mkdir(parents=True)
            print(f'{a} has been created.')
else:
    pass

gisPath = uPath / 'GIS'
lPath = [f for f in gisPath.glob('*')]
netDir = Path(r'\\kcdp-1\KCGIS\MasterGISFiles\Ben')
netDB = netDir / 'GISPro' / 'SDE Connections'

### Only use with notebook

In [ ]:
pd.options.display.max_columns = 40

## Create File GeoDatabase and Feature Datasets

### Create Folders for Parcels with Suffix Data

In [ ]:
#Create Folders for ParcelWithSuffix Data
paFolder = [f for f in lPath if f.name == 'Processing'][0]
paProcessing = paFolder / 'Patriot' / f'{dStr}'
if paProcessing.exists() == True:
    print(f'{paProcessing} already exist.')
else:
    paProcessing.mkdir(parents=True)
    print(f'Created {paProcessing}.')

paFR = [f for f in lPath if f.name == 'Review'][0]
paReview = paFR / 'Patriot' / f'{dStr}'
if paReview.exists() == True:
    print(f'{paReview} already exist')
else:
    paReview.mkdir(parents=True)
    print(f'Created {paReview}')

In [ ]:
iE = netDB / 'MAPPINGADMIN.sde' / 'PROD.MAPPINGADMIN.ParcelEditing'
sr = arc.Describe(f'{iE}').spatialReference
outGDB = gisPath / paFolder / f'Data_{mStr}.gdb'
locGDB = outGDB / f'Daily_{dStr}'
if arc.Exists(f'{outGDB}'):
    print("GDB already exists.")
else:
    arc.CreateFileGDB_management(f'{paFolder}', f'{outGDB.name}')
    print(f'Created File GeoDatabase at {outGDB.parent}')

time.sleep(5)

if arc.Exists(f'{locGDB}'):
    print(f'{locGDB.name} already exists')
else:
    arc.CreateFeatureDataset_management(f'{locGDB.parent}', f'{locGDB.name}', sr)
    print(f'{locGDB.name} Dataset has been created')

## Export Tax Parcels and Addressing FC into LOCAL GDB

### Does not work with Parcel Fabric due to bug with 10.5.1.

In [ ]:
dbAdd = 'PROD.ADDRESSINGADMIN.Addressing'
dbSufP = 'PROD.GISADMIN.Suffix_Parcels'
dbMPTab = 'PROD.gisadmin.MGOPermits'
dbAPTab = 'PROD.gisadmin.All_Permits'

In [ ]:
#create paths to the data instead of using env.workspace makes it so that I can use arc Walk
arc.env.workspace = f'{iE.parent}'

fcList = [dbAdd, dbSufP]
tList = [dbMPTab, dbAPTab]
cParcels = []
cTables = []
for dirpath, dirnames, filenames in arc.da.Walk():
    for f in filenames:
        if f in fcList:
            a  = f.split('.')[-1]
            b = f'{a}_{dStr}'
            if arc.Exists(f'{locGDB / b}'):
                cParcels.append(locGDB.joinpath(b))
                print(f'{b} already exists')
            else:
                arc.FeatureClassToFeatureClass_conversion(f, f'{locGDB}', f'{b}')
                cParcels.append(locGDB.joinpath(b))
                print (f'{b} has been copied')
for dirpath, dirnames, filenames in arc.da.Walk():
    for f in filenames:
        if f in tList:
            a  = f.split('.')[-1]
            b = f'{a}_{dStr}' #Tables need to have a date string since they won't be under the dataset
            # c = locGDB.parent
            if arc.Exists(f'{outGDB / b}'):
                cTables.append(outGDB.joinpath(b))
                print(f'{b} already exists')
            else:
                arc.TableToTable_conversion(f, f'{outGDB}', f'{b}')
                cTables.append(outGDB.joinpath(b))
                print (f'{b} has been copied')
# print(cParcels)
# print(cTables)

# Copying Suffix_Parcels
# cList = ['PROD.GISADMIN.Suffix_Parcels']
# iEPath = list(map(lambda x: iE.parent / x, cList))
# print(f'{iEPath[0]}')

# for f in iEPath:
#     print(f'Copying FCs to {locGDB}')
#     a = str(f).split('.')[-1]
#     b = f'{a}_{dStr}'
#     print(a)
#     g = f'{f}'
#     print(g)
#     arc.FeatureClassToFeatureClass_conversion(g, f'{locGDB}',b)
#     cParcels.append(locGDB / b)
#     print(f'Finished Copying {b} to {locGDB}')

### Works with Parcel Fabric

In [ ]:
#currently using env.workspace to move parcelfabric needs to be changed to use path
pfParcels = 'PROD.MAPPINGADMIN.ParcelFabric_Parcels'
exp = "TYPE = 7 AND Historical = 0"
a = pfParcels.split('.')[-1]
b = f'{a}_{dStr}'
# print(f'{iEPath[1]}')
if arc.Exists(f'{locGDB / b}'):
    cParcels.append(locGDB.joinpath(b))
    print(f'{b} already exists')
else:
    print(f'Copying Parcel Fabric to {locGDB}')
    arc.FeatureClassToFeatureClass_conversion(pfParcels, f'{locGDB}', b, where_clause=exp)
    cParcels.append(locGDB.joinpath(b))
    print(f'Finished Copying {b} to {locGDB}')

### Import Excel Sheet into DataFrame

In [ ]:
pExcel = paProcessing / 'commercial properties.xlsx'
df = pd.read_excel(pExcel)

### Merge the Commerical Properties with the Parcel Fabric layer

Rename the columns to remove whitespace and change Parcel ID to match GIS Parcel #

In [ ]:
df.rename(columns=lambda x: ''.join(x.split()), inplace=True)

In [ ]:
def change(field):
    x = field.split(' ')
    return f'{x[0]}-{x[1]}-{x[2]}-{x[3]}-{x[4]}-{x[5]}01'
df.ParcelID = df.ParcelID.apply(change)

In [ ]:
print(df.loc[df.ParcelID.str.contains('-00001')].shape)
print(df.loc[~df.ParcelID.str.contains('-00001')].shape)

Create DataFrame from Parcel Fabric Polygons

In [ ]:
pfFC = [f for f in cParcels if f.name.startswith('ParcelFabric') == True][0]
ppf_df = features.GeoAccessor.from_featureclass(pfFC, fields=['Name'])
print('Parcel Fabric DataFrame created.')

In [ ]:
ppf_df.rename(columns={'Name': 'ParcelID'}, inplace=True)

In [ ]:
mDf = ppf_df.merge(df, on='ParcelID', how='outer', indicator=True)

In [ ]:
bDf = mDf[mDf._merge == 'both'].copy(deep=True)
bDf.PropertyID = bDf.PropertyID.astype(int)

In [ ]:
bCsv = paReview / 'Both.csv'
bDf.to_csv(bCsv, index=False)

In [ ]:
mCsv = paReview / 'Missing_Parcels.csv'
mDf[(mDf._merge == 'right_only') & (mDf.ParcelID.str.contains('-00001'))].to_csv(mCsv, index=False)

In [ ]:
rCsv = paReview / 'Suffix_Parcels.csv'
rDf = mDf[(mDf._merge == 'right_only') & (~mDf.ParcelID.str.contains('-00001'))].to_csv(rCsv, index=False)

### Create DataFrame from Addressing Feature Class

In [ ]:
#Create Addressing DF from Addressing Feature Class
addFC = [f for f in cParcels if f.name.startswith('Addressing') == True][0]
df_add = features.GeoAccessor.from_featureclass(addFC)

<b>Edit Address DataFrame to remove nulls, correct misnamed Roads, and to remove whitespace</b>

Create new address field using the MSAG columns.

In [ ]:
# df_add['STS'].replace('None', " ")
# df_add.loc[df_add.STS == None, :]
# df_add.isnull().sum()
df_add['STS'].fillna(np.nan, inplace= True)
df_add['STS'].replace(' ', '', inplace=True)
df_add['STS'].replace(np.nan, '', inplace=True)
# df_add['STS'].unique()
# df_add.head()

In [ ]:
aType = df_add.STS.unique()
# print(sType)
for d in aType:
    if d.upper() == 'ROAD':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'RD').strip())
    elif d.upper() == 'DRIVE':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'DR').strip())
        # sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'DR').strip())
    elif d.upper() == 'BOULEVARD':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'BLVD').strip())
        # sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'BLVD').strip())
    elif d.upper() == 'LANE':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'LN').strip())
        # sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'LN').strip())
    elif d.upper() == 'COURT':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'CT').strip())
        # sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'CT').strip())
    elif d.upper() == 'CIRCLE':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'CIR').strip())
        # sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'CIR').strip())
    elif d.upper() == 'STREET':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'ST').strip())
        # sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'ST').strip())
    elif d.upper() == 'HIGHWAY':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'HWY').strip())
        # sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'HWY').strip())
    elif d.upper() == 'AVENUE':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'AVE').strip())
        # sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'AVE').strip())
    elif d.upper() == 'PLACE':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'PL').strip())
        # sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'PL').strip())
    else:
        df_add.STS.str.strip()
# print(df_add.STS.unique())

In [ ]:
df_add.fillna(np.nan, inplace= True)
# df_add.replace(' ', '', inplace=True)
df_add.replace(np.nan, '', inplace=True)
# print(df_add.columns)
# print(df_add.STS.value_counts())

In [ ]:
df_add.SAN = df_add.SAN.astype('str',errors='ignore')
df_add.SAN = df_add.SAN.apply(lambda x: x.split('.')[0])
df_add['FAddress'] = df_add[['SAN', 'PRD', 'STN', 'STS', 'POD']].values.tolist()
# df_add.FAddress.values
df_add['FooAddress'] = df_add['FAddress'].apply(' '.join)
df_add['FoAddress'] = df_add['FooAddress'].apply(lambda x: ' '.join(x.split()))
# df_add

In [ ]:
df_add.FAddress = df_add.FoAddress
kColumns = ['FAddress', 'SHAPE']
# print([x for x in kColumns if x not in kColumns])
df_add.drop(columns=[x for x in df_add if x not in kColumns], inplace=True)
# df_add

In [ ]:
df_add_a = df_add

In [ ]:
ppf_df[ppf_df.ParcelID == '7-02-08520-01-0900-00001']

In [ ]:
bDf[bDf.SHAPE.isnull()]

In [ ]:
pfsr = {'rings': [[[0,0]]], 'spatialReference': {'wkid': 26957, 'latestWkid': 26957}}
bDf.SHAPE = bDf.SHAPE.apply(lambda x: pfsr if x is None else x)

In [ ]:
add_join = bDf.spatial.join(df_add_a)
add_join

### Create Points from Parcel Suffix Feature Class and create DataFrame from the points FC.

In [ ]:
#Create Points out of polygons in Suffix_Parcels
sufPar = [f for f in cParcels if f.name.startswith("Suffix")][0]
# print(sufPar.parent)
# print(cParcels)
sPoints = locGDB / 'sPoints'
# print(sPoints)
if arc.Exists(f'{sPoints}'):
    print(f'{sPoints.name} already exists')
else:
    arc.FeatureToPoint_management(str(sufPar), f'{sPoints}', "INSIDE")
    print(f'{sPoints.name} has been created in {locGDB}')

In [ ]:
#Create DF from Suffix_Parcels
# sFields = [f.name for f in arc.ListFields(sPoints)]
# print(sFields)
sp_df = features.GeoAccessor.from_featureclass(sPoints)
sp_df.drop(columns=['OBJECTID', 'ORIG_FID'], inplace=True)
sp_df.rename(columns={'Name': 'Development', 'Lot':'DevLot'}, inplace=True)
sp_df.rename(columns=lambda x: x.strip().upper(), inplace=True)
print('Suffix Parcels DataFrame created.')

### Merge Pride Tables (Pride Main and AS400 Parcel Data) with Parcels with Suffix Data(sp_df). Export them for Review. 

<b> Left_Only = Values in map_df_a and right_only = Values in sp_df. </b>

In [ ]:
#Merge Pride Table (PrideMain and AS400 Data) and Parcels with Suffix Data (sp_df).
df_Pride_m = map_df_a.merge(sp_df, on='PARCELID', how='outer', indicator=True)
print("Created df_Pride_m from Pride Table and Addressing Data.")
# df_Pride_m.shape

#export the csvs with both being the base data
df_Pride_m[df_Pride_m['_merge'] == 'left_only'].to_csv(psReview / 'Pride_Left.csv', index=False)
df_Pride_m[df_Pride_m['_merge'] == 'right_only'].to_csv(psReview / 'SP_Right.csv', index=False)
df_Pride_m[df_Pride_m['_merge'] == 'both'].to_csv(psReview / 'Pride_Suf_Both.csv', index=False)
print('Exported Pride Table and Suffix Data')

df_Pride_mb is values that matched in the merge betwee Pride Table and Parcels With Suffix

In [ ]:
df_Pride_mb = df_Pride_m[df_Pride_m._merge == 'both'].copy(deep=True)
p_suf_b = len(df_Pride_mb)
# print(p_suf_b)

### Take values not matched from df_Pride_m, which are from original Pride Main and AS400 Parcel Data merge, and try to match them with their address to df_add_a. Export for Review.

df_Pride_L = Values from df_Pride_m(merge between Pride Table and Suffix Parcels) that didn't match.

In [ ]:
df_Pride_L = df_Pride_m[df_Pride_m._merge == 'left_only'].copy(deep=True)
p_suf_l = len(df_Pride_L)
# print(p_suf_l)

Remove Unit from the FLocation Column

In [ ]:
def loc(field):
    a = field.split(', ')
    if a[-1].lower().startswith('unit'):
        return a.pop(0)
    else:
        return field

In [ ]:
df_Pride_L.FLOCATION = df_Pride_L.FLOCATION.apply(loc)

In [ ]:
#remove _merge column
# del df_Pride_m['_merge']
#Unmatched Values from Pride Table (Pride and AS400 Data) merged with Addressing Data
df_Pride_L.drop(columns=['_merge', 'SHAPE'] , inplace=True)
df_Pride_all = df_Pride_L.merge(df_add_a, left_on='FLOCATION', right_on='FAddress', how='outer', indicator=True)
# print(df_Pride_all.shape)
print('Created df_Pride_all from Pride Table and Addressing Data.')

<b> Left_Only = Values in df_Pride_all. Right_Only = Values in df_Pride_L.

In [ ]:
#export csvs to see what needs to be fixed both needs to be appended to df_Pride_m
df_Pride_all[df_Pride_all['_merge'] == 'left_only'].to_csv(psReview / 'ASP_Add_Left.csv', index=False)
df_Pride_all[df_Pride_all['_merge'] == 'both'].to_csv(psReview / 'ASP_Add_Both.csv', index=False)
df_Pride_all[df_Pride_all['_merge'] == 'right_only'].to_csv(psReview / 'ASP_Add_Right.csv', index=False)
print('Exported Pride Table and Address Table.')

<b> df_Pride_ab is values that matched between Pride, Suffix Merge and Addressing Data. </b>

In [ ]:
df_Pride_ab = df_Pride_all[df_Pride_all._merge == 'both'].copy(deep=True)
p_add_b = len(df_Pride_ab)
# print(p_add_b)

In [ ]:
df_Pride_al = df_Pride_all[df_Pride_all._merge == 'left_only'].copy(deep=True)
p_add_l = len(df_Pride_al)
# print(p_add_l)

loc_df are values that don't start with a number. geo_df are values that start with a number to make it easier to geocode.

In [ ]:
loc_df = df_Pride_al[~df_Pride_al.FLOCATION.str.contains('^[0-9]+')].copy(deep=True)
geo_df = df_Pride_al[df_Pride_al.FLOCATION.str.contains('^[0-9]+')].copy(deep=True)

<b> Added PARENT Column to match with Parcel Fabric DataFrame, edited addressing to match addressing DataFrame, updated the columns names, created column (OriginalAdd) to maintain the non-edited address. </b>

In [ ]:
def change(field):
    a = field.split('-')
    return f'{a[0]}-{a[1]}-{a[2]}-{a[3]}-{a[4]}-00001'

loc_df['PARENT']= loc_df.PARCELID.apply(change)
geo_df['PARENT'] = geo_df.PARCELID.apply(change)

Clear up column names in loc_df and geo_df

In [ ]:
loc_df.dropna(axis=1, how='all', inplace=True)
loc_df.drop(columns='_merge', inplace=True)

In [ ]:
geo_df.dropna(axis=1, how='all', inplace=True)
geo_df.drop(columns='_merge', inplace=True)

### CSV with manual edits to get address data into correct format. Created from ASP_Add_Left output. Do not use for automation since the parcels could change in the future.

In [ ]:
#Create df from edits CSV
pDir = netDir / 'ParcelWithSuffix' / 'Edits_ASP_Add_Left.csv'
cLocal = psReview / pDir.name
print(f'Copying {pDir.name} to {cLocal.parent}')
copy2(pDir, cLocal)
print(f'Finished Copying {cLocal.name} to {cLocal.parent}')
df_Pride_edits = pd.read_csv(cLocal)
df_Pride_edits.shape

In [ ]:
df_Pride_edits.columns

In [ ]:
df_Pride_edits = df_Pride_edits.rename(columns={'Unnamed: 7' : 'OriginalAdd'})

In [ ]:
print(df_add_a.columns)
print(df_Pride_edits.columns)

<b> Merge Addressing with edits </b>

In [ ]:
#merge addressing with df_Pride_edits
df_Pride_Add = df_Pride_edits.merge(df_add_a, left_on='UPPER_ADDR', right_on='Address', how='outer', indicator=True)
# df_Pride_Add

Left_Only = Values from df_Pride_edits. Right_Only = Values from df_add

In [ ]:
df_Pride_Add[df_Pride_Add['_merge'] == 'left_only'].to_csv(psReview / 'Ed_Add_Left.csv', index=False)
df_Pride_Add[df_Pride_Add['_merge'] == 'both'].to_csv(psReview / 'Ed_Add_Both.csv', index=False)
df_Pride_Add[df_Pride_Add['_merge'] == 'right_only'].to_csv(psReview / 'Ed_Add_Right.csv', index=False)
print('Exported Pride Table Edits and Address Table.')

In [ ]:
df_Pride_Add.loc[df_Pride_Add._merge == 'left_only']
#  & df_Pride_Add.FullAddr.str.contains('1679')
df_Pride_AddL = df_Pride_Add.loc[df_Pride_Add._merge == 'left_only']
df_Pride_AddB = df_Pride_Add.loc[df_Pride_Add._merge == 'both']
print(df_Pride_AddL.shape)
print(df_Pride_AddB.shape)


In [ ]:
# df_add.loc[(df_add.STN.str.contains('KENTWOOD', na=False)) & (df_add['SAN'].astype(str).str.contains('23', na=False))]

In [ ]:
# df_Pride_AddL.drop(columns='_merge', inplace=True)
# df_Pride_AddL.drop(columns='SHAPE', inplace=True)
df_Pride_AddL.columns

### Create Points from Parcel Fabric for geospatial information for Addresses and Parcels with Suffixes that don't match

In [ ]:
#need to not rely on arc.env.workspace for outputs.
pfFC = [f for f in cParcels if f.name.startswith('ParcelFabric') == True][0]
# print(pfFC)
pfPoints = locGDB / f'pfPoints_{dStr}'
# print(pfPoints)
if arc.Exists(f'{pfPoints}'):
    print(f'{pfPoints.name} already exists.')
else:
    arc.FeatureToPoint_management(str(pfFC), f'{pfPoints}', "INSIDE")
    print(f'{pfPoints.name} has been created.')

In [ ]:
# Create DataFrame from pfPoints FC
pfP_df = features.GeoAccessor.from_featureclass(pfPoints, fields=['Name'])
print('Parcel Fabric Points DataFrame created.')
# pfP_df.shape
# pfP_df.columns

#Use if need to edit field names in pfPoints FC
# pfFields = [f.name for f in arc.ListFields(pfPoints)]
# print(pfFields)

<b> Attempt to geocode the final addresses. Can use OSM Nominatim, ESRI AGOL (uses credits from org), FirstMap Locator on their REST Server, or KC batch geocoder on REST Server.</b>

In [ ]:
# del Add_Par_df['_merge']
# Add_Par_df = df_Pride_AddL.merge(pfP_df, how='inner', left_on='Fu', right_on='Name')
# Add_Par_df

In [ ]:
# Add_Par_df[Add_Par_df.Name.duplicated()]

In [ ]:
# df_Pride_AddL[df_Pride_AddL.Fu.duplicated()]

In [ ]:
# addPardf = pReview / 'AddParDF.csv'
# Add_Par_df.to_csv(addPardf, index=False)

In [ ]:
from arcgis.gis import GIS
from arcgis.gis.server import Server
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())
pURL = os.getenv('PORTAL_SITE')
aURL = os.getenv('AGOL_SITE')
sURL = os.getenv('STAGE_SITE')
sUser = os.getenv('STAGE_USERNAME')
sPass = os.getenv('STAGE_PASSWORD')
aUser = os.getenv('AGOL_USERNAME')
aPass = os.getenv('AGOL_PASSWORD')
pUser = os.getenv('ESRI_USERNAME')
pPass = os.getenv('ESRI_PASSWORD')

#Connect to Enterprise GIS
gisE = GIS(url=pURL, username=pUser, password=pPass)

#Connect to AGOL
gisA = GIS(url=aURL, username=aUser, password=aPass, set_active=False)

#Connect to Staging. Does not work for right now. Found a solution: https://community.esri.com/t5/developers-questions/error-connecting-to-arcgis-server-with-arcgis-python-api/m-p/869042?commentID=896496#comment-896496

# gisT = GIS(url=sURL, username=sUser, password=sPass, set_active=False, verify_cert=False)
# from arcgis.gis.server import Server
# sURL = 'https://ESRISTAGE:6443'
# gisT = Server(url=f'{sURL}/ed8fb7ce-e99d-4eb2-af78-b7ef11e3c2c4/admin',token_url=f'{sURL}/ed8fb7ce-e99d-4eb2-af78-b7ef11e3c2c4/tokens/generateToken', username=sUser, password=sPass, verify_cert=False, set_active=False)

#This method for connecting to staging server does work. Must use Server Module for now
gisT = Server(url=f'{sURL}/arcgis/admin', username=sUser, password=sPass, verify_cert=False)

# gisT = GIS(url=f'{sURL}/arcgis/admin', username=sUser, password=sPass, verify_cert=False)


print(gisA)
print(gisE)
print(gisT)

In [ ]:
# from arcgis.gis.server import ServerManager

# gisT.datastores.add_database()

# kServ = gisE.admin.servers.list()[0]
# kServ.datastores.search()

In [ ]:
def addz(field, field1):
    if field1 != ' ':
        return f'{field}, {field1}'
    else:
        return field
geo_df['ZLOCATION'] = geo_df.apply(lambda x: addz(x['FLOCATION'], x['ZIPCODE']), axis=1)

In [ ]:
#Geocoding from_df is very buggy, changed the _accessor.py located at C:\Users\MKinnaman\Anaconda3\envs\arcNew
#\Lib\site-packages\arcgis\features\geo. Added Line 2247 to 2250 and Line 2261.

from arcgis import geocoding
from arcgis.geocoding import Geocoder
# from arcgis.features import GeoAccessor

locFM = 'https://enterprise.firstmap.delaware.gov/arcgis/rest/services/Location/Delaware_FirstMap_Locator/GeocodeServer'
geoc = Geocoder(locFM)

# print(geoc.properties.locatorProperties.MaxBatchSize)

# def addST(field):
#     return field + ', DOVER, DE'
# df_Pride_AddL['GAddress'] = df_Pride_AddL.UPPER_ADDR.apply(addST)
# df_Pride_AddL


# if len(geo_df.ZLOCATION) <= 499:
#     aList = geo_df.ZLOCATION.values.tolist()
# else:
#     clist = [aList[i:i + 499] for i in range(0, len(aList), 499)]

# add_geom = []
# geo_b = {}
# if aList is not None:
#     geo = geocoding.batch_geocode(aList, out_sr=26957, geocoder= geoc)
#     add_geom.append(geo)
# else:
#     j = len(clist)
#     for i in range(0, j):
#       geo = geocoding.batch_geocode(clist[i], )

geo_B = features.GeoAccessor.from_df(geo_df, address_column='ZLOCATION', geocoder=geoc, sr=26957)
geo_B

Create CSV of geocoded addresses.

In [ ]:
geo_add = psReview / 'Geo_Add.csv'
geo_add_df = geo_B[geo_B.SHAPE.notnull()].copy(deep=True)
geo_add_df.to_csv(geo_add)

In [ ]:
geo_match = len(geo_add_df)
# print(geo_match)

Get the values that were not matched after geocoding and merge them with the Parcel ID from Parcel Fabric Points (pfP_df)

In [ ]:
geo_b_null = geo_B[geo_B.SHAPE.isnull()].copy(deep=True)
del geo_b_null['SHAPE']
geo_pf = geo_b_null.merge(pfP_df, left_on='PARENT', right_on='Name', how='outer', indicator=True)

In [ ]:
dl = psReview / 'MissingParcels.csv'
geo_pf.loc[geo_pf._merge == 'left_only', :].to_csv(dl, index=False)

In [ ]:
geo_pf_df = geo_pf[geo_pf._merge == 'both'].copy(deep=True)
geo_pf_b = len(geo_pf_df)
# print(geo_pf_b)

Merge parcels with an address with Parcel Fabric Points.

In [ ]:
pl = psReview / 'MissingParcels_2.csv'
loc_pf = loc_df.merge(pfP_df, left_on='PARENT', right_on='Name', how='outer', indicator=True)
loc_pf.loc[loc_pf._merge == 'left_only', :].to_csv(pl, index=False)

In [ ]:
loc_pf_df = loc_pf.loc[loc_pf._merge == 'both', :]
loc_pf_b = len(loc_pf_df)
# print(loc_pf_b)

In [ ]:
ls = [p_suf_b, p_add_b, geo_pf_b, loc_pf_b, geo_match]
sum(ls)

<b>Concat all the DFs into one. All of the DFs need to have the same columns. df_Pride_mb, df_pride_ab, geo_pf_df, loc_pf_df, and geo_add_df</b>

In [ ]:
print(geo_pf_df.columns)
print(df_Pride_ab.columns)
print(df_Pride_mb.columns)
print(loc_pf_df.columns)
print(geo_add_df.columns)

In [ ]:
geo_pf_df.drop(columns=['_merge','Name'], inplace=True)
gList = geo_pf_df.columns.tolist()
# gList.remove('_merge')
# kCol = ['PARCELID']
# print(aClist)
# df_Pride_ab.drop(columns=[x for x not in gList], inplace=True)
# map_df_a.columns
df_Pride_ab.drop(columns=[x for x in df_Pride_ab.columns if x not in gList], inplace=True)
df_Pride_mb.drop(columns=[x for x in df_Pride_mb.columns if x not in gList], inplace=True)
loc_pf_df.drop(columns=[x for x in loc_pf_df.columns if x not in gList], inplace=True)
geo_add_df.drop(columns=[x for x in geo_add_df.columns if x not in gList], inplace=True)

Add PARENT column to all of the DFs for use with MGO Model.

In [ ]:
df_Pride_ab['PARENT'] = df_Pride_ab.PARCELID.apply(change)
df_Pride_mb['PARENT'] = df_Pride_mb.PARCELID.apply(change)

In [ ]:
frames = [geo_pf_df, df_Pride_ab, df_Pride_mb, loc_pf_df, geo_add_df]
df_Append_all = pd.concat(frames, axis=0)
cDF = psReview / 'ParcelWithSuffix.csv'
df_Append_all.to_csv(cDF, index=False)

Edit the column names for clarity

In [ ]:
# oCols = [f.lower() for f in df_Append_all.columns]
rCols = {'PARCELID': 'Name', 'FLOCATION' : 'Address'}
dCols = ['BILLINGADDRESS', 'BILLINGADDRESS2', 'ZIPCODE', 'FORMATTEDLOCATION', 'ZLOCATION']
df_Append_all.drop(columns=dCols, inplace=True)
df_Append_all.rename(columns=rCols, inplace=True)
# df_Append_all.rename(columns=lambda x: x.title(), inplace=True)
# df_Append_all.rename(columns=lambda x: x.upper(), inplace=True)

In [ ]:
df_Append_all.columns

In [ ]:
df_Append_all.reset_index(drop=True, inplace=True)

Testing for bugs in arcgis package

In [ ]:
# outc = psReview / 'all.csv'
# df_Append_all.to_csv(outc)

In [ ]:
# print(geo_pf_df.spatial.sr)
# print(df_Pride_ab.spatial.sr)
# print(df_Pride_mb.spatial.sr)
# print(loc_pf_df.spatial.sr)
# print(geo_add_df.spatial.sr)
# print(df_Append_all.spatial.sr)
# print(pfP_df.spatial.sr)
# print(geo_B.spatial.sr)
# outFC = locGDB / f'geo_Test'
# geo_pf_df.spatial.to_featureclass(f'{outFC}', sanitize_columns=False)
# from uuid import uuid4
# import random
# import string
# for x in frames:
#     outFC = locGDB / f'{random.choice(string.ascii_lowercase)}'
#     a = pd.DataFrame(x)
#     a.spatial.to_featureclass(location=f'{outFC}', sanitize_columns=False)

# frames = [geo_pf_df, df_Pride_ab, df_Pride_mb, loc_pf_df, geo_add_df]

Output the DataFrame to the geodatabase

In [ ]:
outFC = locGDB / f'ParcelWithSuffix_{dStr}'
df_Append_all.spatial.to_featureclass(f'{outFC}', sanitize_columns=False)
print(f'Copied {outFC.name} to {locGDB}')

Publish the DataFrame to Portal for use with online mapping.

In [ ]:
df_Append_all.spatial.to_featurelayer('ParcelWithSuffix', gis= gisE)
print('Added ParcelWithSuffix to Portal')